In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import os as os
import fetchyc
import collections

# 1.Fetch and combine all available data into single data frame table

In [ ]:
srcdir = r'.\data\raw\YC'
destdir = r'.\'

In [36]:
FirstConversion()

.\data\raw\YC\EU\France\France 1-Month Bond Yield Historical Data (1).csv
.\data\raw\YC\EU\France\France 1-Month Bond Yield Historical Data.csv
.\data\raw\YC\EU\France\France 1-Year Bond Yield Historical Data (1).csv
.\data\raw\YC\EU\France\France 1-Year Bond Yield Historical Data.csv
.\data\raw\YC\EU\France\France 10-Year Bond Yield Historical Data (1).csv
.\data\raw\YC\EU\France\France 10-Year Bond Yield Historical Data.csv
.\data\raw\YC\EU\France\France 15-Year Bond Yield Historical Data (1).csv
.\data\raw\YC\EU\France\France 15-Year Bond Yield Historical Data.csv
.\data\raw\YC\EU\France\France 2-Year Bond Yield Historical Data (1).csv
.\data\raw\YC\EU\France\France 2-Year Bond Yield Historical Data.csv
.\data\raw\YC\EU\France\France 20-Year Bond Yield Historical Data (1).csv
.\data\raw\YC\EU\France\France 20-Year Bond Yield Historical Data.csv
.\data\raw\YC\EU\France\France 25-Year Bond Yield Historical Data.csv
.\data\raw\YC\EU\France\France 3-Month Bond Yield Historical Data (1).

.\data\raw\YC\EU\United Kingdom\United Kingdom 8-Year Bond Yield Historical Data (1).csv
.\data\raw\YC\EU\United Kingdom\United Kingdom 8-Year Bond Yield Historical Data (2).csv
.\data\raw\YC\EU\United Kingdom\United Kingdom 8-Year Bond Yield Historical Data.csv
.\data\raw\YC\EU\United Kingdom\United Kingdom 9-Year Bond Yield Historical Data (1).csv
.\data\raw\YC\EU\United Kingdom\United Kingdom 9-Year Bond Yield Historical Data (2).csv
.\data\raw\YC\EU\United Kingdom\United Kingdom 9-Year Bond Yield Historical Data.csv
.\data\raw\YC\NA\USA\USA Gov_TreasureConstantMaturity_Nominal.csv


In [121]:





dataset = tf.data.experimental.make_csv_dataset('./EU_Germany_12046.csv',batch_size = 14000, shuffle = True)
rr = iter(dataset.take(1).map(lambda x: mapelement(x))).next()
rr
            

Tensor("concat_16:0", shape=(2, 238001), dtype=float32)


{'Date': <tf.Tensor: shape=(), dtype=string, numpy=b'2012-01-05'>,
 'XY': <tf.Tensor: shape=(2, 135456), dtype=float32, numpy=
 array([[5.650e+00, 9.000e-03, 3.540e+00, ..., 4.479e+00, 4.126e+00,
         4.417e+00],
        [3.000e+00, 3.000e+00, 3.000e+00, ..., 3.600e+02, 3.600e+02,
         3.600e+02]], dtype=float32)>}

In [ ]:
def mapelement(table):
    valpairs = None
    for key, x in TermMapToMonth.items():
        if key in not table:
            continue
        xy = table[key][table[key] > 256.0]
        if table[key].dtype == tf.string:
            print(table[key])
            #xy = tf.strings.to_number(table[key][table[key] != b''], out_type=tf.dtypes.float32)
            xy = tf.strings.to_number(table[key], out_type=tf.dtypes.float32)
            print(xy)
            print(table[key])
        if xy.shape[0] is not None:
            xy = tf.stack([xy, tf.repeat(x,xy.shape[0])])
            if valpairs is None:
                valpairs = xy
            else:
                valpairs = tf.concat([valpairs,xy], axis = 1)       
    m = table['Date'].shape[0] // 2
    return {'Date' : table['Date'][m], 'XY' : valpairs}

@tf.function
def mapelement1(table):
    obss = tf.convert_to_tensor(np.array([[-256.0], [-256.0]]), dtype = tf.float32)
    tt = 'M01'
    x = TermMapToMonth[tt]
    xy = tf.convert_to_tensor(table[tt])
    print(xy)
    lx = xy > -255.0
    xy = tf.boolean_mask(xy, lx)
    #print(xy)
    #print(xy.numpy())
    xy = tf.stack([xy, tf.repeat(x,xy.shape[0])])
    obss = tf.concat([obss,xy], axis = 1) 
    print(obss)
    lx = obss[0] > -255.0
    print(lx)
    obss = tf.boolean_mask(obss, lx, axis = 1)
    median = table['Date'].shape[0] // 2
    return {'Date' : table['Date'][median], 'XY' : xy}

In [ ]:
t = np.linspace(6.5, 20.0, 5)
#t = [6.8]
b0 = 5
b1 = -1
b2 = 3
b0, b1, b2 = (0.0466, -0.0429, 0.0712)
for x in np.linspace(0.0, 100.0, 10):
    b0 = np.linspace(0.0, 100.0, 1000)
    m = 6.5
    y = np.exp(b0) + ((b1 + b2) * (1.0 - np.exp(-x/m))/(x/m)) - b2 * np.exp(-x/m)
    plt.plot(b0, y)
plt.show()

In [ ]:
    
b0 = np.linspace(-100, 100.0, 100)
b2 = 3.14
for b1 in np.linspace(0.0, 17.5, 10):
    y = (np.exp(b0) + np.exp(b1 - b0) - b2)**2
    plt.plot(b0, y)
plt.show()